In [1]:
# import tensorflow as tf
# from keras.models import Sequential
# from keras.layers import Conv2D,MaxPooling2D,Dense,Dropout,Flatten,BatchNormalization,Activation
# from keras.optimizers import Adam 
# from keras.preprocessing.image import ImageDataGenerator


import os
import shutil
import numpy as np
import seaborn as sns
import pandas as pd
import tensorflow as tf

from matplotlib import pyplot as plt
from tensorflow.keras.utils import load_img, img_to_array, array_to_img
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.efficientnet import EfficientNetB5
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import Input, Conv2D, concatenate, MaxPooling2D, Add, AveragePooling2D, MaxPool2D
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Flatten, Activation
from tensorflow.keras.activations import relu, softmax
from tensorflow.keras.optimizers import Adam, Adadelta
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model, save_img
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras import initializers

from sklearn.metrics import confusion_matrix
import itertools


In [38]:
train_datagen = ImageDataGenerator(#rotation_range = 180,
                                         width_shift_range = 0.1,
                                         height_shift_range = 0.1,
                                         horizontal_flip = True,
                                         rescale = 1./255,
                                         #zoom_range = 0.2,
                                         validation_split = 0.2
                                        )
validation_datagen = ImageDataGenerator(rescale = 1./255,
                                         validation_split = 0.8)

In [39]:
# train_generator=train_data_gen.flow_from_directory(
#     'train',
#     target_size=(48,48),
#     batch_size=64,
#     color_mode='grayscale',
#     class_mode='categorical'
    
# )

# validation_generator=validation_data_gen.flow_from_directory(
#     'val',
#     target_size=(48,48),
#     batch_size=64,
#     color_mode='grayscale',
#     class_mode='categorical'
    
# )




train_generator = train_datagen.flow_from_directory(directory = 'train',
                                                    target_size = (48,48),
                                                    batch_size = 64,
                                                    color_mode = "grayscale",
                                                    class_mode = "categorical",
                                                    subset = "training"
                                                   )
validation_generator = validation_datagen.flow_from_directory( directory = 'val',
                                                              target_size = (48,48),
                                                              batch_size = 64,
                                                              color_mode = "grayscale",
                                                              class_mode = "categorical",
                                                              subset = "validation"
                                                             )

Found 26118 images belonging to 8 classes.
Found 6529 images belonging to 8 classes.


In [35]:
# model=Sequential()
# model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(48,48,1)))
# model.add(Conv2D(64,kernel_size=(3,30),activation='relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.25))

# model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
# model.add(Dropout(0.25))

# model.add(Flatten())
# model.add(Dense(1024,activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(7,activation='softmax'))
# # model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001,decay=1e-6),metrics=['accuracy'])


In [40]:
model = Sequential()

# 1 convalutional
# model.add(Conv2D(64, (5, 5), padding='same', input_shape=(48, 48, 1), kernel_initializer="he_normal"))
model.add(Conv2D(64, (3, 3), padding='same', input_shape=(48, 48, 1)))
model.add(BatchNormalization(axis=3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 2 convalutional
# model.add(Conv2D(128, (3, 3), padding='same', kernel_initializer="he_normal"))
model.add(Conv2D(128, (5, 5), padding='same'))
model.add(BatchNormalization(axis=3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 3 convalutional
# model.add(Conv2D(256, (3, 3), padding='same', kernel_initializer="he_normal"))
model.add(Conv2D(256, (3, 3), padding='same'))
model.add(BatchNormalization(axis=3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 4 convalutional
# model.add(Conv2D(512, (3, 3), padding='same', kernel_initializer="he_normal"))
model.add(Conv2D(512, (3, 3), padding='same'))
model.add(BatchNormalization(axis=3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


model.add(Flatten())
# model.add(Dense(256, kernel_initializer="he_normal"))
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

# model.add(Dense(512, kernel_initializer="he_normal"))
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(8, activation='softmax'))

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 48, 48, 64)        640       
                                                                 
 batch_normalization_24 (Bat  (None, 48, 48, 64)       256       
 chNormalization)                                                
                                                                 
 activation_24 (Activation)  (None, 48, 48, 64)        0         
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 24, 24, 64)       0         
 g2D)                                                            
                                                                 
 dropout_24 (Dropout)        (None, 24, 24, 64)        0         
                                                                 
 conv2d_17 (Conv2D)          (None, 24, 24, 128)      

In [41]:
plot_model(model,
           show_shapes=True,
           show_layer_names=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [42]:
# img_size = 48
# batch_size = 64

# datagen_train = ImageDataGenerator(horizontal_flip=True)
# train_gen = datagen_train.flow_from_directory("train",
#                                                     target_size=(img_size, img_size),
#                                                     color_mode='grayscale',
#                                                     batch_size=batch_size,
#                                                     class_mode='categorical',
#                                                     shuffle=True)

# datagen_validation = ImageDataGenerator(horizontal_flip=True)
# validation_gen = datagen_train.flow_from_directory('val',
#                                                     target_size=(img_size, img_size),
#                                                     color_mode='grayscale',
#                                                     batch_size=batch_size,
#                                                     class_mode='categorical',
#                                                     shuffle=True)

In [43]:
# reduce_lr = ReduceLROnPlateau(monitor='val_loss',
#                               factor=0.1,
#                               patience=2,
#                               min_lr=0.00001,
#                               model='auto')

# check_point = ModelCheckpoint(filepath='FERmodified_net3.hdf5',
#                               monitor='val_loss',
#                               save_best_only=True,
#                               mode='min',
#                               verbose=1)

# early_stop = EarlyStopping(monitor='val_loss',
#                            patience=5,
#                            restore_best_weights=True)

# callbacks = [reduce_lr, early_stop]

In [44]:
model.compile(optimizer=Adam(learning_rate=0.0005),
              loss=categorical_crossentropy,
              metrics=['accuracy'])

In [45]:
# history = model.fit(train_gen,
#                       epochs=30,
#                       callbacks=callbacks,
#                       validation_data=validation_gen)

In [46]:
optimiser = tf.keras.optimizers.Adam(lr=0.0001)
model.compile(optimizer=optimiser,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 48, 48, 64)        640       
                                                                 
 batch_normalization_24 (Bat  (None, 48, 48, 64)       256       
 chNormalization)                                                
                                                                 
 activation_24 (Activation)  (None, 48, 48, 64)        0         
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 24, 24, 64)       0         
 g2D)                                                            
                                                                 
 dropout_24 (Dropout)        (None, 24, 24, 64)        0         
                                                                 
 conv2d_17 (Conv2D)          (None, 24, 24, 128)      

In [49]:
epochs = 10
batch_size = 64

In [51]:
history=model.fit(x=train_generator,epochs=20,validation_data=train_generator)

Epoch 1/20
409/409 [==============================] - 353s 863ms/step - loss: 1.1079 - accuracy: 0.5790 - val_loss: 1.2582 - val_accuracy: 0.4975
Epoch 2/20
409/409 [==============================] - 329s 804ms/step - loss: 1.0886 - accuracy: 0.5897 - val_loss: 1.0209 - val_accuracy: 0.6125
Epoch 3/20
409/409 [==============================] - 363s 887ms/step - loss: 1.0741 - accuracy: 0.5920 - val_loss: 1.0763 - val_accuracy: 0.5925
Epoch 4/20
409/409 [==============================] - 339s 829ms/step - loss: 1.0561 - accuracy: 0.6012 - val_loss: 1.1013 - val_accuracy: 0.5842
Epoch 5/20
409/409 [==============================] - 373s 913ms/step - loss: 1.0427 - accuracy: 0.6073 - val_loss: 1.0770 - val_accuracy: 0.5973
Epoch 6/20
409/409 [==============================] - 434s 1s/step - loss: 1.0297 - accuracy: 0.6098 - val_loss: 1.1409 - val_accuracy: 0.5789
Epoch 7/20
409/409 [==============================] - 424s 1s/step - loss: 1.0138 - accuracy: 0.6188 - val_loss: 1.1805 - val_a

In [ ]:
# #Training first 4 Epcohs and saving
# model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=32, epochs=4)
# model.save("partial.h5")


# #loading the model, training another 4 Epochs and then saving the updated model.
# from keras.models import load_model
# new_model = load_model('partial.h5')
# new_model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=32, initial_epoch=4,epochs=8)
# new_model.save("updated.h5")

In [52]:
model.save('30epoch.h5')

In [53]:
new_model=load_model('30epoch.h5')

In [54]:
history=model.fit(x=train_generator,initial_epoch=20,epochs=31,validation_data=train_generator)

Epoch 21/31
409/409 [==============================] - 353s 863ms/step - loss: 0.8736 - accuracy: 0.6761 - val_loss: 0.8811 - val_accuracy: 0.6657
Epoch 22/31
409/409 [==============================] - 346s 846ms/step - loss: 0.8645 - accuracy: 0.6750 - val_loss: 0.8830 - val_accuracy: 0.6675
Epoch 23/31
409/409 [==============================] - 338s 826ms/step - loss: 0.8669 - accuracy: 0.6747 - val_loss: 0.9057 - val_accuracy: 0.6520
Epoch 24/31
409/409 [==============================] - 342s 837ms/step - loss: 0.8407 - accuracy: 0.6892 - val_loss: 0.7763 - val_accuracy: 0.7032
Epoch 25/31
409/409 [==============================] - 342s 836ms/step - loss: 0.8378 - accuracy: 0.6825 - val_loss: 1.1751 - val_accuracy: 0.5776
Epoch 26/31
409/409 [==============================] - 337s 824ms/step - loss: 0.8315 - accuracy: 0.6883 - val_loss: 0.7884 - val_accuracy: 0.6976
Epoch 27/31
409/409 [==============================] - 327s 799ms/step - loss: 0.8258 - accuracy: 0.6924 - val_loss: 1

In [55]:
model.save('newmp.h5')

In [56]:
history=model.fit(x=train_generator,initial_epoch=31,epochs=38,validation_data=train_generator)

Epoch 32/38
409/409 [==============================] - 345s 844ms/step - loss: 0.8053 - accuracy: 0.6999 - val_loss: 0.8258 - val_accuracy: 0.6890
Epoch 33/38
409/409 [==============================] - 370s 905ms/step - loss: 0.7964 - accuracy: 0.7046 - val_loss: 0.8048 - val_accuracy: 0.6948
Epoch 34/38
409/409 [==============================] - 340s 832ms/step - loss: 0.7860 - accuracy: 0.7083 - val_loss: 0.8868 - val_accuracy: 0.6680
Epoch 35/38
409/409 [==============================] - 340s 831ms/step - loss: 0.7719 - accuracy: 0.7105 - val_loss: 0.7532 - val_accuracy: 0.7165
Epoch 36/38
409/409 [==============================] - 340s 831ms/step - loss: 0.7748 - accuracy: 0.7085 - val_loss: 0.7448 - val_accuracy: 0.7223
Epoch 37/38
409/409 [==============================] - 345s 844ms/step - loss: 0.7714 - accuracy: 0.7096 - val_loss: 0.8847 - val_accuracy: 0.6659
Epoch 38/38
409/409 [==============================] - 342s 836ms/step - loss: 0.7696 - accuracy: 0.7113 - val_loss: 0

In [57]:
model.save('third.h5')

In [58]:
history=model.fit(x=train_generator,initial_epoch=39,epochs=45,validation_data=train_generator)

Epoch 40/45
409/409 [==============================] - 378s 924ms/step - loss: 0.7538 - accuracy: 0.7197 - val_loss: 0.6289 - val_accuracy: 0.7723
Epoch 41/45
409/409 [==============================] - 340s 830ms/step - loss: 0.7525 - accuracy: 0.7219 - val_loss: 0.7988 - val_accuracy: 0.7044
Epoch 42/45
409/409 [==============================] - 354s 866ms/step - loss: 0.7567 - accuracy: 0.7193 - val_loss: 0.6474 - val_accuracy: 0.7612
Epoch 43/45
409/409 [==============================] - 492s 1s/step - loss: 0.7451 - accuracy: 0.7206 - val_loss: 0.8899 - val_accuracy: 0.6719
Epoch 44/45
409/409 [==============================] - 415s 1s/step - loss: 0.7412 - accuracy: 0.7233 - val_loss: 0.6890 - val_accuracy: 0.7413
Epoch 45/45
409/409 [==============================] - 354s 864ms/step - loss: 0.7349 - accuracy: 0.7267 - val_loss: 0.7325 - val_accuracy: 0.7210


In [59]:
from keras.models import model_from_json

In [63]:
model_json=model.to_json()
with open('model.json','w') as json_file:
    json_file.write(model_json)


In [61]:
model.save_weights('modelweight.h5')